## Setup

In [21]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D, Bidirectional, GRU
from tensorflow.keras import Model

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import math

print(tf.__version__)

2.10.1


In [22]:
def get_model(image_channels=3, vgg_weights_path=None):
    image_shape = (None, None, image_channels)  # 大小不定
    base_model = VGG16(weights=None, include_top=False, input_shape=image_shape)
    if vgg_weights_path is not None:  # 基础模型预训练微调
        base_model.load_weights(vgg_weights_path)
        base_model.trainable = True
    else:
        base_model.trainable = False

    # 基础模型输入和输出
    input = base_model.input
    sub_output = base_model.get_layer('block5_conv3').output

    # tf.print(sub_output.shape)

    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', name='rpn_conv1')(sub_output)
    # x1 = Lambda(_reshape, output_shape=(None, 512))(x)  # output_shape not include batch dim, (N*H, W, C), C=512
    # b = tf.shape(x)
    # x1 = tf.reshape(x, [b[0] * b[1], b[2], b[3]])  # (N x H, W, C)
    x1 = tf.squeeze(x, axis=0)
    # x1.set_shape((None, None, 512))
    # x1 = tf.ensure_shape(x1, (31, 31, 512))

    x2 = Bidirectional(GRU(128, return_sequences=True, reset_after=False), name='blstm')(x1)
    b = tf.shape(x)
    x3 = tf.reshape(x2, [b[0], b[1], b[2], 256])  # (N, H, W, 256)
    x3 = Conv2D(512, (1, 1), padding='same', activation='relu', name='lstm_fc')(x3)

    # 分类分支
    cls = Conv2D(10 * 2, (1, 1), padding='same', activation='linear', name='rpn_class_origin')(x3)
    # cls = Lambda(_reshape3, output_shape=(None, 2), name='rpn_class')(cls)  # (N, H*W*10, 2)
    b = tf.shape(cls)
    cls = tf.reshape(cls, [b[0], b[1] * b[2] * 10, 2], name='rpn_class')  # (N, H x W x 10, 2)
    # cls_prod = Activation('softmax', name='rpn_cls_softmax')(cls)

    # 高度回归分支
    regr = Conv2D(10 * 2, (1, 1), padding='same', activation='linear', name='rpn_regress_origin')(x3)
    # regr = Lambda(_reshape3, output_shape=(None, 2), name='rpn_regress')(regr)  # (N, H*W*10, 2)
    b = tf.shape(regr)
    regr = tf.reshape(regr, [b[0], b[1] * b[2] * 10, 2], name='rpn_regress')  # (N, H x W x 10, 2)

    # predict_model = Model(input, [cls, regr, cls_prod])

    train_model = Model(input, [cls, regr])

    return train_model


In [23]:
model = get_model()

model.summary()

input_image = tf.random.normal(shape=(1, 500, 500, 3), dtype=tf.float32)

print(model(input_image))

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   1792        ['input_4[0][0]']                
                                64)                                                               
                                                                                                  
 block1_conv2 (Conv2D)          (None, None, None,   36928       ['block1_conv1[0][0]']           
                                64)                                                         

                                                                 0]']                             
                                                                                                  
 lstm_fc (Conv2D)               (None, None, None,   131584      ['tf.reshape_9[0][0]']           
                                512)                                                              
                                                                                                  
 rpn_class_origin (Conv2D)      (None, None, None,   10260       ['lstm_fc[0][0]']                
                                20)                                                               
                                                                                                  
 rpn_regress_origin (Conv2D)    (None, None, None,   10260       ['lstm_fc[0][0]']                
                                20)                                                               
          

## Convert the Keras model to TensorFlow Lite

In [24]:
export_path = "./saved_model"
tf.saved_model.save(model, export_path)

INFO:tensorflow:Assets written to: ./saved_model\assets


INFO:tensorflow:Assets written to: ./saved_model\assets


In [25]:
# Convert Keras model to TF Lite format.

tflite_path = "./ctpn.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TFLite ops
    tf.lite.OpsSet.SELECT_TF_OPS,  # enable TF ops
]
tflite_model = converter.convert()
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

In [26]:
import os
model_path = os.path.join(os.getcwd(), 'ctpn.tflite')
print('TF Lite model:', model_path)

TF Lite model: C:\Users\Lenovo\Desktop\mt-io-dev\ocr\ctpn\ctpn.tflite


## Verify the TensorFlow Lite model

In [27]:
# Run inference with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_path=model_path)

input_details = interpreter.get_input_details()
interpreter.resize_tensor_input(input_details[0]['index'], (1, 500, 500, 3), strict=True)
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]["index"], input_image)

print(input_details)

interpreter.invoke()
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])()[0]
print(output)

[{'name': 'serving_default_input_4:0', 'index': 0, 'shape': array([1, 1, 1, 3]), 'shape_signature': array([-1, -1, -1,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[-0.00084487 -0.00101603]
 [-0.00069394 -0.00203349]
 [ 0.00082251  0.00164492]
 ...
 [ 0.00117558  0.00314122]
 [ 0.00234521  0.00219317]
 [-0.00174341 -0.00112847]]
